In [3]:
#Tweets 2 - Immigrants

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import plotly.offline as py
#For this library it was required to install plotly and geopandas via Conda, also update setuptools and install ez_setup
import plotly.figure_factory as ff  
import vincent
from geopy.geocoders import Nominatim
import xlrd

#Configuration files
from config import gkey
py.offline.init_notebook_mode(connected=True)

In [3]:
tweets = pd.ExcelFile("Resources/TweetData/2nd_Search_Twitter_Posts.xls")
df = tweets.parse("searchd")
df['Longitude'] = ''
df['Latitude'] = ''
df.head(5)

,GUID,Date (EST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,...,Following,Post Title,Post Type,Image URL,Object,Scene,Action,Brand,Longitude,Latitude
0,930303273579532288,2017-11-14 00:15:59,http://twitter.com/___success___/status/930303...,RT @jamKartel White Man: NIGGER Y'all: We need...,@___success___,Cuianna🌺,United States of America,NaN,NaN,Neutral,...,245,RT @jamKartel White Man: NIGGER\nY'all: We nee...,Retweet,NaN,NaN,NaN,NaN,NaN,,
1,799081689788071936,2016-11-16 21:48:15,http://twitter.com/brianbethel/status/79908168...,If @culturemap is #altright and #blacklivesmat...,@brianbethel,brian bethel,United States of America,Texas,Dallas,Neutral,...,422,If @culturemap is #altright and #blacklivesmat...,Tweet,NaN,NaN,NaN,NaN,NaN,,
2,831281371540754432,2017-02-13 18:18:18,http://twitter.com/LesIzmoreKC/status/83128137...,RT @Drugg Me: White ppl r annoying *White pers...,@LesIzmoreKC,Octavia Butler's SUN,United States of America,Missouri,Kansas City,Negative,...,886,RT @Drugg Me: White ppl r annoying\n\n*White p...,Retweet,NaN,NaN,NaN,NaN,NaN,,
3,851610353024864256,2017-04-10 21:38:24,http://twitter.com/JeffBain6/status/8516103530...,RT @jojoh888 .We have immigration laws for a r...,@JeffBain6,Jeff Bain,United States of America,Texas,NaN,Negative,...,122,RT @jojoh888 .We have immigration laws for a r...,Retweet,NaN,NaN,NaN,NaN,NaN,,
4,722247394423939072,2016-04-18 22:16:12,http://twitter.com/CorinaLankford/status/72224...,RT @VAC4NT_ just a lonely nigger.,@CorinaLankford,Corina Lankford,United States of America,NaN,NaN,Negative,...,201,RT @VAC4NT_ just a lonely nigger.,Retweet,NaN,NaN,NaN,NaN,NaN,,


In [ ]:
#Variables to be used during data collection
processed = 0
notfound = 0
count = 0

#Loop to collect Lat/Long for Agency Names
for (idx, row) in df.iterrows():
    try:
        #Store current row value for each column
        city = (row.loc['City/Urban Area'])
        state = (row.loc['State/Region'])
        #URL from Google APIs where Lat/Long values will be collected
        query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city},{state}&key={gkey}"
        #JSON request
        response = requests.get(query_url).json()
        #Variables to store lat/long values 
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        #Lat/Long columns created and respective values stored in current row
        df.at[idx, 'Latitude'] = lat
        df.at[idx, 'Longitude'] = lng
        #Print to verify data is processing as expected
        print(f"{city},{state}:{lat},{lng}")
        #Count to validate how many city names have been processed
        processed += 1
        #Error handling if some of the city names are not found
    except:
        #Print to verify when an city is not found
        print("City not found")
        #Count to validate how many city names were not found
        notfound += 1
        continue
#Print to validate final count for both processed and not found agency names
print(f"{processed} agencies were processed. {notfound} agencies were not found.")


nan,nan:36.103234,-95.87159059999999
Dallas,Texas:32.7766642,-96.79698789999999
Kansas City,Missouri:39.0997265,-94.5785667
nan,Texas:33.1980612,-96.61613229999999
nan,nan:36.103234,-95.87159059999999
nan,nan:36.103234,-95.87159059999999
San Bernardino,California:34.1083449,-117.2897652
nan,New Hampshire:42.7711577,-71.4115492
nan,nan:36.103234,-95.87159059999999
nan,Michigan:39.94414099999999,-86.2374366
Bridgeport,Connecticut:41.1792258,-73.1894384
nan,nan:36.103234,-95.87159059999999
nan,nan:36.103234,-95.87159059999999
nan,nan:36.103234,-95.87159059999999
Houston,Texas:29.7604267,-95.3698028
nan,California:38.257725,-122.0666346
nan,nan:36.103234,-95.87159059999999
nan,Florida:27.913083,-82.5056544
nan,Texas:33.1980612,-96.61613229999999
nan,nan:36.103234,-95.87159059999999
nan,Arizona:33.6548218,-112.3552682
nan,nan:36.103234,-95.87159059999999
Austin,Texas:30.267153,-97.7430608
New York,New York:36.1023715,-115.1745559
nan,Missouri:38.9491861,-92.32857039999999
San Francisco,Cali

In [ ]:
# Run Vader Analysis on each tweet
for (idx, row) in output_list.iterrows():
    text = (row.loc['text_tweet'])
    counter += 1
    results = analyzer.polarity_scores(text)
    compound = results["compound"]
    pos = results["pos"]
    neu = results["neu"]
    neg = results["neg"]
    tweet_times.append(tweet["created_at"])
    output_list.at[idx, 'compound'] = compound
    output_list.at[idx, 'pos'] = pos
    output_list.at[idx, 'neu'] = neu
    output_list.at[idx, 'neg'] = neg
